In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 그래프 기본 테마 설정
# https://coldbrown.co.kr/2023/07/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%8B%A4%EC%A0%84%ED%8E%B8-08-seaborn-sns-set%EC%9D%84-%ED%86%B5%ED%95%B4-%EC%8A%A4%ED%83%80%EC%9D%BC-%EC%84%A4%EC%A0%95%ED%95%98%EA%B8%B0/
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False


# 복잡한 통계 처리를 위한 라이브러리
from scipy import stats

In [2]:
df1 = pd.read_csv('../data/total_real_final.csv')
df1

,고객ID,위도,경도,성별,고령자여부,배우자여부,부양가족여부,가입개월수,전화서비스가입여부,복수회선여부,...,프리미엄기술지원여부,음악스트리밍이용여부,무제한데이터이용여부,총환불액,총초과데이터요금,총장거리통화요금,총납부금,이탈유형,한글이탈유형,한글이탈이유
0,0002-ORFBO,34.827662,-118.999073,False,False,True,False,9,True,False,...,True,False,True,0.00,0,381.51,974.81,NaN,NaN,NaN
1,0003-MKNFE,34.162515,-118.203869,True,False,False,False,9,True,True,...,False,True,False,38.33,10,96.21,610.28,NaN,NaN,NaN
2,0004-TLHLJ,33.645672,-117.922613,True,False,False,False,4,True,False,...,False,False,True,0.00,0,134.60,415.45,Competitor,경쟁사,가격이 너무 높음
3,0011-IGKFF,38.014457,-122.115432,True,True,True,False,13,True,False,...,False,False,True,0.00,0,361.66,1599.51,Dissatisfaction,불만족,제품 불만족
4,0013-EXCHZ,34.227846,-119.079903,False,True,True,False,3,True,False,...,True,False,True,0.00,0,22.14,289.54,Dissatisfaction,불만족,네트워크 안정성
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,9987-LUTYD,32.759327,-116.997260,False,False,False,False,13,True,False,...,True,True,True,0.00,0,606.84,1349.74,NaN,NaN,NaN
7039,9992-RRAMN,37.734971,-120.954271,True,False,True,False,22,True,True,...,False,True,True,0.00,0,356.40,2230.10,Dissatisfaction,불만족,제품 불만족
7040,9992-UJOEL,39.108252,-123.645121,True,False,False,False,2,True,False,...,False,False,True,0.00,0,37.24,129.99,NaN,NaN,NaN
7041,9993-LHIEB,33.001813,-117.263628,True,False,True,False,67,True,False,...,True,True,True,0.00,0,142.04,4769.69,NaN,NaN,NaN


In [3]:
# reason_mapping 딕셔너리를 한글 키로 변환 (한글 → 한글)
reason_mapping_kor = {
    '경쟁사가 더 나은 제안': '경쟁사가 더 나은 제안',
    '이동했습니다.': '이동했습니다.',
    '경쟁사가 더 나은 기기를 보유': '경쟁사가 더 나은 기기를 보유',
    '경쟁사가 더 빠른 다운로드 속도를 제공': '경쟁사가 더 빠른 다운로드 속도를 제공',
    '경쟁사가 더 많은 데이터를 제공': '경쟁사가 더 많은 데이터를 제공',
    '가격이 너무 높음': '가격이 너무 높다.',
    '제품 불만족': '제품 불만족',
    '서비스 불만족': '서비스 불만족',
    '웹사이트의 셀프 서비스 부족': '웹사이트의 셀프 서비스 부족',
    '네트워크 안정성': '네트워크 안정성',
    '제한된 서비스 범위': '제한된 서비스 범위',
    '합리적인 가격의 다운로드/업로드 속도가 부족': '합리적인 가격의 다운로드/업로드 속도가 부족',
    '장거리 요금': '장거리 요금',
    '추가 데이터 요금': '추가 데이터 요금',
    '모르겠다.': '모르겠다.',
    '온라인 지원의 전문성 부족': '온라인 지원의 전문성 부족',
    '전화 지원의 전문성 부족': '전화 지원의 전문성 부족',
    '서비스 제공업체의 태도': '서비스 제공업체의 태도',
    '지원 담당자의 태도': '지원 담당자의 태도',
    '사망': '사망'
}

# 한글이탈이유 컬럼 대체
df1['한글이탈이유'] = df1['한글이탈이유'].replace(reason_mapping_kor)

In [4]:
grouped_reasons = (
    df1.groupby(['한글이탈유형', '한글이탈이유'])
    .size()
    .reset_index(name='이탈이유_갯수')
    .sort_values(['한글이탈유형', '이탈이유_갯수'], ascending=[True, False])
)

# 출력
grouped_reasons

,한글이탈유형,한글이탈이유,이탈이유_갯수
0,가격,가격이 너무 높다.,78
10,가격,추가 데이터 요금,40
6,가격,장거리 요금,35
7,가격,저렴한 다운로드/업로드 속도 부족,31
3,가격,모르겠다,6
5,가격,웹사이트의 셀프 서비스 부족,5
8,가격,제품 불만족,5
1,가격,경쟁사가 더 나은 기기를 보유했습니다,4
4,가격,서비스 제공업체의 태도,3
9,가격,지원 담당자의 태도,3


In [5]:
df1['한글이탈이유'].value_counts()

한글이탈이유
지원 담당자의 태도                192
경쟁사가 더 빠른 다운로드 속도를 제공함    189
경쟁사가 더 많은 데이터를 제공함        162
모르겠다                      154
경쟁사가 더 나은 제안을 했습니다        140
서비스 제공업체의 태도              135
경쟁사가 더 나은 기기를 보유했습니다      130
네트워크 안정성                  103
제품 불만족                    102
가격이 너무 높다.                 98
서비스 불만족                    89
웹사이트의 셀프 서비스 부족            88
추가 데이터 요금                  57
이동했습니다                     53
제한된 서비스 범위                 44
저렴한 다운로드/업로드 속도 부족         44
장거리 요금                     44
전화 지원의 전문성 부족              20
온라인 지원의 전문성 부족             19
사망                          6
Name: count, dtype: int64

In [45]:
df1['한글이탈이유'] = df1['한글이탈이유'].replace('경쟁사가 더 나은 제안을 했습니다', '경쟁사가 더 나은 제안을 함')
df1['한글이탈이유'] = df1['한글이탈이유'].replace('경쟁사가 더 나은 기기를 보유했습니다', '경쟁사가 더 나은 기기를 보유함')
df1['한글이탈이유'] = df1['한글이탈이유'].replace('가격이 너무 높다.', '가격이 너무 높음')
df1['한글이탈이유'] = df1['한글이탈이유'].replace('모르겠다', '모르겠음')

In [51]:
# Step 1. 네가 정리한 유형-이유 mapping
reason_to_type2 = {
    '경쟁사가 더 빠른 다운로드 속도를 제공함': '경쟁사',
    '경쟁사가 더 많은 데이터를 제공함': '경쟁사',
    '경쟁사가 더 나은 제안을 함': '경쟁사',
    '경쟁사가 더 나은 기기를 보유함': '경쟁사',
    '지원 담당자의 태도': '태도',
    '서비스 제공업체의 태도': '태도',
    '제품 불만족': '불만족',
    '네트워크 안정성': '불만족',
    '서비스 불만족': '불만족',
    '제한된 서비스 범위': '불만족',
    '웹사이트의 셀프 서비스 부족': '불만족',
    '온라인 지원의 전문성 부족': '불만족',
    '전화 지원의 전문성 부족': '불만족',
    '가격이 너무 높음': '가격',
    '추가 데이터 요금': '가격',
    '장거리 요금': '가격',
    '저렴한 다운로드/업로드 속도 부족': '가격',
    '모르겠음': '기타',
    '이동했습니다': '기타',
    '사망': '기타'
}

# Step 2. 데이터프레임에 이탈유형2 컬럼 생성
df1['한글이탈유형2'] = df1['한글이탈이유'].map(reason_to_type2)

In [53]:
# Step 3. 파일로 저장
output_file = '../data/total_real_final2.csv'
df1.to_csv(output_file, index=False, encoding='utf-8-sig')  # 한글 깨짐 방지

print(f"CSV 파일이 성공적으로 저장되었습니다: {output_file}")

CSV 파일이 성공적으로 저장되었습니다: ../data/total_real_final2.csv


#### 서비스별 월요금차이

In [72]:
# 분석할 서비스 리스트
services = [
    '온라인보안서비스여부',
    '온라인백업서비스여부',
    '기기보호서비스여부',
    '기술지원서비스여부',
    'TV스트리밍이용여부',
    '영화스트리밍이용여부',
    '음악스트리밍이용여부'
]

# 결과를 저장할 데이터프레임 초기화
service_fees = pd.DataFrame()

# 각 서비스별로 평균 월요금 계산
for service in services:
    avg_fee = df1.groupby(service)['월요금'].mean().reset_index().rename(columns={service: '가입여부', '월요금': '평균_월요금'})
    avg_fee['서비스'] = service
    service_fees = pd.concat([service_fees, avg_fee], ignore_index=True)

# 서비스 컬럼 순서 조정
service_fees = service_fees[['서비스', '가입여부', '평균_월요금']]

# 결과 출력
service_fees

,서비스,가입여부,평균_월요금
0,온라인보안서비스여부,False,59.104538
1,온라인보안서비스여부,True,78.838732
2,온라인백업서비스여부,False,55.117339
3,온라인백업서비스여부,True,83.081597
4,기기보호서비스여부,False,54.247371
5,기기보호서비스여부,True,84.822254
6,기술지원서비스여부,False,58.252931
7,기술지원서비스여부,True,80.680137
8,TV스트리밍이용여부,False,49.793877
9,TV스트리밍이용여부,True,88.736738


In [76]:
# 가입여부가 True인 것만 필터링
filtered_fees = service_fees[service_fees['가입여부'] == True]

# 서비스 컬럼을 가로로 pivot
summary_df = filtered_fees.pivot_table(
    index=None, 
    columns='서비스', 
    values='평균_월요금'
)

# index를 없애고 reset_index()로 데이터프레임화
summary_df.reset_index(drop=True, inplace=True)

summary_df

서비스,TV스트리밍이용여부,기기보호서비스여부,기술지원서비스여부,영화스트리밍이용여부,온라인백업서비스여부,온라인보안서비스여부,음악스트리밍이용여부
0,88.736738,84.822254,80.680137,88.475714,83.081597,78.838732,86.59375


In [80]:
# CSV로 내보내기
summary_df.to_csv('../data/서비스별가입자평균월요금.csv', index=False, encoding='utf-8-sig')